Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
KEYWORDS = ['python', 'парсинг','html','']
URL = 'https://habr.com/ru/all/'

In [2]:
res=requests.get(URL)

In [3]:
soup = BeautifulSoup(res.text, 'html.parser')

In [4]:
posts = soup.find_all('article', class_='post')

In [5]:
def texter(URLt):
    req=requests.get(URLt)
    soup = BeautifulSoup(req.text, 'html.parser')
    #print(soup)
    posts = soup.find('div',class_="post__text").find_all('p')
    if(posts==[]):
        posts=soup.find('div',class_="post__text").text
        return(posts)
    a=""
    for post in posts:
        a+=post.text
    return(a)
#print(texter())

In [6]:

data=pd.DataFrame({'Дата':[],'Заголовок':[],'Ссылка':[],'Текст статьи':[]})
i=0
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
            if any([hub_lower in desired for desired in KEYWORDS]):
                title_element = post.find('a', class_='post__title_link')
                time=post.find('span',class_='post__time')
                data.loc[i]=[time.text,title_element.text, title_element.attrs.get('href'),texter(title_element.attrs.get('href'))]
                i+=1
data

,Дата,Заголовок,Ссылка,Текст статьи
0,сегодня в 17:15,Как посмотреть плоские фильмы в 3D,https://habr.com/ru/post/529602/,Это расширенная версия моей публикации на Medi...
1,сегодня в 16:07,MMORPG больше не в Telegram — Swift и Kotlin —...,https://habr.com/ru/post/529660/,Прошло полгода с выпуска пробной статьи про со...
